In [1]:

import pandas as pd
import numpy as np

#image tools
from matplotlib import pyplot as plt
from PIL import Image

#filesystem tools that allow for file manipulation
import os
from glob import glob

#ML Stuff



import tensorflow as tf

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
with tf.device('/gpu:0'):
    graph = tf.Graph()
    with graph.as_default():

      # Variables.
      batch_size = 128  #mini batch for SGD
      h1_size=1024  #number of neurons in hidden layer
      beta = .001  #regularization 
      keep_prob=.5 #probability of dropout
      image_size = 28
      num_labels = 10


      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(mnist.validation.images)
      tf_test_dataset = tf.constant(mnist.test.images)

      # weights and biases for hidden layer 1
      w_h1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, h1_size]))
      b_h1 = tf.Variable(tf.zeros([h1_size]))

      # weights and biases for output/logit layer
      w_logit = tf.Variable(
        tf.truncated_normal([h1_size, num_labels]))
      b_logit = tf.Variable(tf.zeros([num_labels]))

      # keep probability for dropout.   this is a variable so it can be turned on for train and off for predict
      keep_prob = tf.placeholder("float")


      def model(data):
            """
            Assembles the NN
            """
            h1 = tf.nn.relu(tf.matmul(data, w_h1) + b_h1)
            h1_drop_out = tf.nn.dropout(h1, keep_prob)
            return tf.matmul(h1_drop_out, w_logit) + b_logit  #return the output layer


      # Training Computations
      logits = model(tf_train_dataset)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
      regularized_loss = tf.nn.l2_loss(w_h1) + tf.nn.l2_loss(w_logit)
      total_loss = loss + beta * regularized_loss

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(total_loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
      test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
num_steps = 5001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
 
    # Generate a minibatch.
    batch_data, batch_labels = mnist.train.next_batch(batch_size)


    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : .5}

    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={keep_prob:1.0}), mnist.validation.labels.astype(float)))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob:1.0}), mnist.test.labels))

Initialized
Minibatch loss at step 0: 359.579590
Minibatch accuracy: 11.7%
Validation accuracy: 28.2%
